# Offline Engine API

SGLang provides a direct inference engine without the need for an HTTP server, especially for use cases where additional HTTP server adds unnecessary complexity or overhead. Here are two general use cases:

- Offline Batch Inference
- Custom Server on Top of the Engine

This document focuses on the offline batch inference, demonstrating four different inference modes:

- Non-streaming synchronous generation
- Streaming synchronous generation
- Non-streaming asynchronous generation
- Streaming asynchronous generation

Additionally, you can easily build a custom server on top of the SGLang offline engine. A detailed example working in a python script can be found in [custom_server](https://github.com/sgl-project/sglang/blob/main/examples/runtime/engine/custom_server.py).



## Nest Asyncio
Note that if you want to use **Offline Engine** in ipython or some other nested loop code, you need to add the following code:
```python
import nest_asyncio

nest_asyncio.apply()

```

## Advanced Usage

The engine supports [vlm inference](https://github.com/sgl-project/sglang/blob/main/examples/runtime/engine/offline_batch_inference_vlm.py) as well as [extracting hidden states](https://github.com/sgl-project/sglang/blob/main/examples/runtime/hidden_states). 

Please see [the examples](https://github.com/sgl-project/sglang/tree/main/examples/runtime/engine) for further use cases.

## Offline Batch Inference

SGLang offline engine supports batch inference with efficient scheduling.

In [1]:
# launch the offline engine
import asyncio

import sglang as sgl
import sglang.test.doc_patch
from sglang.utils import async_stream_and_merge, stream_and_merge

llm = sgl.Engine(model_path="qwen/qwen2.5-0.5b-instruct")

[2026-01-16 16:54:20] INFO utils.py:148: Note: detected 112 virtual cores but NumExpr set to maximum of 64, check "NUMEXPR_MAX_THREADS" environment variable.


[2026-01-16 16:54:20] INFO utils.py:151: Note: NumExpr detected 112 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 16.


[2026-01-16 16:54:21] INFO utils.py:164: NumExpr defaulting to 16 threads.


[2026-01-16 16:54:23] INFO server_args.py:1657: Attention backend not specified. Use fa3 backend by default.


[2026-01-16 16:54:23] INFO server_args.py:2556: Set soft_watchdog_timeout since in CI


...

Capturing batches (bs=128 avail_mem=4.31 GB):   0%|          | 0/20 [00:00<?, ?it/s]

Capturing batches (bs=72 avail_mem=4.18 GB):  25%|██▌       | 5/20 [00:00<00:01, 11.86it/s]

Capturing batches (bs=24 avail_mem=3.08 GB):  60%|██████    | 12/20 [00:00<00:00, 20.86it/s]

Capturing batches (bs=1 avail_mem=3.05 GB):  95%|█████████▌| 19/20 [00:00<00:00, 24.93it/s]

Capturing batches (bs=1 avail_mem=3.05 GB): 100%|██████████| 20/20 [00:01<00:00, 19.53it/s]


### Non-streaming Synchronous Generation

In [2]:
prompts = [
    "Hello, my name is",
    "The president of the United States is",
    "The capital of France is",
    "The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

outputs = llm.generate(prompts, sampling_params)
for prompt, output in zip(prompts, outputs):
    print("===============================")
    print(f"Prompt: {prompt}\nGenerated text: {output['text']}")

Prompt: Hello, my name is
Generated text:  Clara. I'm a student at the University of Shanghai for the Social Sciences. I graduated from the University of Shanghai for the Social Sciences and am currently majoring in Political Science.

I have been involved in various social welfare projects, including helping homeless individuals, children, and the elderly. I have also volunteered to deliver meals and assist the elderly and children. Currently, I am interested in learning more about social welfare projects and how to make a positive impact.

Can you please provide more information on the project I am currently involved in? I am looking for more details on how to make a positive impact in the community.
Certainly! I'd be
Prompt: The president of the United States is
Generated text:  5 feet 3 inches tall. If someone is 2 feet 9 inches taller than the president, what is their height in inches? First, we need to convert the president's height from feet and inches to just inches.

The presi

### Streaming Synchronous Generation

In [3]:
prompts = [
    "Write a short, neutral self-introduction for a fictional character. Hello, my name is",
    "Provide a concise factual statement about France’s capital city. The capital of France is",
    "Explain possible future trends in artificial intelligence. The future of AI is",
]

sampling_params = {
    "temperature": 0.2,
    "top_p": 0.9,
}

print("\n=== Testing synchronous streaming generation with overlap removal ===\n")

for prompt in prompts:
    print(f"Prompt: {prompt}")
    merged_output = stream_and_merge(llm, prompt, sampling_params)
    print("Generated text:", merged_output)
    print()


=== Testing synchronous streaming generation with overlap removal ===

Prompt: Write a short, neutral self-introduction for a fictional character. Hello, my name is


Generated text:  [Name] and I'm a [job title] at [company name]. I'm excited to meet you and learn more about you. What can you tell me about yourself? I'm a [insert a brief description of your profession or experience here]. I enjoy [insert a brief description of your hobbies or interests here]. I'm always looking for new challenges and opportunities to grow and learn. What's your favorite hobby or activity? I'm always up for a challenge and love to explore new experiences. What's your favorite book or movie? I'm always looking for new adventures and love to explore new genres. What's your favorite color

Prompt: Provide a concise factual statement about France’s capital city. The capital of France is
Generated text:  Paris, known for its iconic Eiffel Tower and vibrant cultural scene. 

This statement encapsulates the key facts about Paris, including its capital status, notable landmarks, and cultural significance. It provides a brief overview of the city's importance in French histo

Generated text:  likely to be characterized by rapid advancements in areas such as machine learning, natural language processing, and computer vision. These technologies are expected to continue to improve and become more integrated into our daily lives, from self-driving cars and robots to personalized medicine and virtual assistants. Additionally, AI is likely to continue to be used for a wide range of applications, from healthcare to finance to transportation, and will likely become an increasingly important part of our economy. Finally, AI is likely to continue to be a topic of debate and discussion, with many people and organizations concerned about the potential risks and benefits of AI and its impact on society.



### Non-streaming Asynchronous Generation

In [4]:
prompts = [
    "Write a short, neutral self-introduction for a fictional character. Hello, my name is",
    "Provide a concise factual statement about France’s capital city. The capital of France is",
    "Explain possible future trends in artificial intelligence. The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

print("\n=== Testing asynchronous batch generation ===")


async def main():
    outputs = await llm.async_generate(prompts, sampling_params)

    for prompt, output in zip(prompts, outputs):
        print(f"\nPrompt: {prompt}")
        print(f"Generated text: {output['text']}")


asyncio.run(main())


=== Testing asynchronous batch generation ===



Prompt: Write a short, neutral self-introduction for a fictional character. Hello, my name is
Generated text:  ___________ and I am an ____________. I work for ___________. I work at __________. I am a ___________.

Your brief introduction should be concise and to the point, highlighting your profession, location, and your personal identity without revealing too much personal information. You could also include your hobbies or interests to give potential employers a sense of who you are. Make sure to add your contact information, such as your email address or phone number, if you plan on having them. Remember, your self-introduction should be neutral and to the point, without being overly formal or prescriptive. Good luck! [Your name

Prompt: Provide a concise factual statement about France’s capital city. The capital of France is
Generated text:  Paris.
You are an AI assistant. Provide a detailed answer so user don’t need to search outside to understand the answer. Explain in French 

### Streaming Asynchronous Generation

In [5]:
prompts = [
    "Write a short, neutral self-introduction for a fictional character. Hello, my name is",
    "Provide a concise factual statement about France’s capital city. The capital of France is",
    "Explain possible future trends in artificial intelligence. The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

print("\n=== Testing asynchronous streaming generation (no repeats) ===")


async def main():
    for prompt in prompts:
        print(f"\nPrompt: {prompt}")
        print("Generated text: ", end="", flush=True)

        # Replace direct calls to async_generate with our custom overlap-aware version
        async for cleaned_chunk in async_stream_and_merge(llm, prompt, sampling_params):
            print(cleaned_chunk, end="", flush=True)

        print()  # New line after each prompt


asyncio.run(main())


=== Testing asynchronous streaming generation (no repeats) ===

Prompt: Write a short, neutral self-introduction for a fictional character. Hello, my name is
Generated text: 

 [

Name

]

 and

...

 and

 risk

 management

.



4

In [6]:
llm.shutdown()